# Object Cropper for Floorplan Dataset

Interactive OpenCV tool for creating cropped object datasets from floorplan images.

## Purpose
Manually crop objects from floorplan images and save them into structured label folders for training deep learning models (e.g. Stable Diffusion, DETR, SAM).

## Supported Labels
- `backgrounds`
- `1stdoor`
- `2sdoor`
- `stove`
- `toilet`
- `sink`
- `closet`

## Controls
- **Mouse Drag** – Select crop area  
- **b** – Save as `backgrounds`  
- **1** – Save as `1stdoor`  
- **2** – Save as `2sdoor`  
- **s** – Save as `stove`  
- **t** – Save as `toilet`  
- **k** – Save as `sink`  
- **c** – Save as `closet`  
- **SPACE** – Next image  
- **q** – Quit  

## Output Structure
```text
train_data/
├── backgrounds/
├── 1stdoor/
├── 2sdoor/
├── stove/
├── toilet/
├── sink/
└── closet/


In [ ]:
import cv2
import os
import glob
import numpy as np

# ================= CONFIG =================
INPUT_FOLDER = r"Data\images_before"   # תיקיית המקור
OUTPUT_FOLDER = "Data\Label_Pics_for_Synthetic_Data_Generation"                        # תיקיית היעד
CROP_SIZE = (512, 512)                              # גודל התוצאה הסופית
DISPLAY_WIDTH = 1000                                # רוחב התמונה בחלון התצוגה (פיקסלים)
# ==========================================

# --------- LABELS / KEYS CONFIG ----------
# NOTE: "doors" renamed to "2sdoor"
LABEL_KEYS = {
    'b': "backgrounds",
    '1': "1stdoor",
    '2': "2sdoor",
    's': "stove",
    't': "toilet",
    'k': "sink",     # k for sinK (כדי לא להתנגש עם s של stove)
    'c': "closet",
}

# הבהובים שונים לכל קטגוריה (BGR)
FLASH_COLOR = {
    "backgrounds": (0, 255, 0),   # ירוק
    "1stdoor":     (255, 255, 0), # צהבהב
    "2sdoor":      (255, 0, 0),   # כחול (כמו שהיה לך לדלת)
    "stove":       (0, 165, 255), # כתום
    "toilet":      (128, 0, 128), # סגול
    "sink":        (0, 255, 255), # צהוב
    "closet":      (0, 0, 255),   # אדום
}
# -----------------------------------------

# יצירת תיקיות פלט לכל לייבל
label_dirs = {}
for key, label in LABEL_KEYS.items():
    out_dir = os.path.join(OUTPUT_FOLDER, label)
    os.makedirs(out_dir, exist_ok=True)
    label_dirs[label] = out_dir

# משתנים גלובליים
drawing = False
ix, iy = -1, -1
img_display = None
img_raw = None
current_crop = None
scale_factor = 1.0

def get_next_filename(folder, prefix):
    """מוצא את המספר הפנוי הבא לשמירה"""
    existing = glob.glob(os.path.join(folder, f"{prefix}_*.png"))
    if not existing:
        return f"{prefix}_01.png"

    nums = []
    for f in existing:
        try:
            name = os.path.basename(f)
            num = int(name.split('_')[1].split('.')[0])
            nums.append(num)
        except:
            pass

    if not nums:
        return f"{prefix}_01.png"

    next_num = max(nums) + 1
    return f"{prefix}_{next_num:02d}.png"

def flash_preview(img, color_bgr, win="Preview (512x512)", ms=200):
    """אפקט הבהוב קצר"""
    if img is None:
        return
    flash = np.full_like(img, color_bgr)
    cv2.imshow(win, cv2.addWeighted(img, 0.7, flash, 0.3, 0))
    cv2.waitKey(ms)
    cv2.imshow(win, img)

def draw_rect(event, x, y, flags, param):
    global ix, iy, drawing, img_display, img_raw, current_crop, scale_factor

    if event == cv2.EVENT_LBUTTONDOWN:
        drawing = True
        ix, iy = x, y

    elif event == cv2.EVENT_MOUSEMOVE:
        if drawing:
            img_copy = img_display.copy()
            cv2.rectangle(img_copy, (ix, iy), (x, y), (0, 255, 0), 2)
            cv2.imshow("Data Cropper", img_copy)

    elif event == cv2.EVENT_LBUTTONUP:
        drawing = False
        cv2.rectangle(img_display, (ix, iy), (x, y), (0, 255, 0), 2)
        cv2.imshow("Data Cropper", img_display)

        # המרת קואורדינטות מתצוגה לתמונה מקורית
        orig_ix = int(ix / scale_factor)
        orig_iy = int(iy / scale_factor)
        orig_x  = int(x / scale_factor)
        orig_y  = int(y / scale_factor)

        x_start, x_end = min(orig_ix, orig_x), max(orig_ix, orig_x)
        y_start, y_end = min(orig_iy, orig_y), max(orig_iy, orig_y)

        if x_end - x_start > 10 and y_end - y_start > 10:
            h, w = img_raw.shape[:2]
            x_start = max(0, x_start)
            y_start = max(0, y_start)
            x_end   = min(w, x_end)
            y_end   = min(h, y_end)

            crop = img_raw[y_start:y_end, x_start:x_end]

            if crop.size > 0:
                current_crop = cv2.resize(crop, CROP_SIZE, interpolation=cv2.INTER_AREA)
                cv2.imshow("Preview (512x512)", current_crop)
                print(f"\n✅ Selected area: {x_start},{y_start} to {x_end},{y_end}")
                print_instructions()

def print_instructions():
    print("Save crop as:")
    # הדפסה מסודרת לפי קטגוריות/מפתחות
    for k, label in LABEL_KEYS.items():
        print(f"  '{k}' -> {label}")
    print("Other:")
    print("  SPACE -> next image")
    print("  q     -> quit")
    print("Crop again anytime by dragging mouse.\n")

def save_current_crop_as(label_name):
    global current_crop
    if current_crop is None:
        return
    out_dir = label_dirs[label_name]
    prefix = label_name.rstrip('s')  # background(s) -> background, אבל לא קריטי
    fname = get_next_filename(out_dir, prefix)
    save_path = os.path.join(out_dir, fname)
    cv2.imwrite(save_path, current_crop)
    print(f"💾 Saved {label_name.upper()}: {fname}")

    color = FLASH_COLOR.get(label_name, (0, 255, 0))
    flash_preview(current_crop, color)

def main():
    global img_display, img_raw, current_crop, scale_factor

    extensions = ['*.jpg', '*.png', '*.jpeg']
    files = []
    for ext in extensions:
        files.extend(glob.glob(os.path.join(INPUT_FOLDER, ext)))

    print(f"Found {len(files)} images. Starting tool...")
    print(f"Display width set to: {DISPLAY_WIDTH}px\n")
    print_instructions()

    cv2.namedWindow("Data Cropper")
    cv2.setMouseCallback("Data Cropper", draw_rect)

    for f_path in files:
        img_raw = cv2.imread(f_path)
        current_crop = None
        if img_raw is None:
            continue

        # שינוי גודל לתצוגה
        h, w = img_raw.shape[:2]
        scale_factor = DISPLAY_WIDTH / float(w)
        new_height = int(h * scale_factor)
        img_display = cv2.resize(img_raw, (DISPLAY_WIDTH, new_height), interpolation=cv2.INTER_AREA)

        cv2.imshow("Data Cropper", img_display)
        print(f"\nProcessing: {os.path.basename(f_path)}")

        while True:
            key = cv2.waitKey(1) & 0xFF
            if key == 255:  # no key
                continue

            # שמירה לפי מקשים שהוגדרו
            key_chr = chr(key).lower() if key < 128 else ''
            if key_chr in LABEL_KEYS:
                label = LABEL_KEYS[key_chr]
                save_current_crop_as(label)

            elif key == ord(' '):
                print(">>> Next Image")
                break

            elif key == ord('q'):
                print("Exiting...")
                cv2.destroyAllWindows()
                return

    cv2.destroyAllWindows()
    print("Done!")

if __name__ == "__main__":
    main()


Found 455 images. Starting tool...
Display width set to: 1000px

Save crop as:
  'b' -> backgrounds
  '1' -> 1stdoor
  '2' -> 2sdoor
  's' -> stove
  't' -> toilet
  'k' -> sink
  'c' -> closet
Other:
  SPACE -> next image
  q     -> quit
Crop again anytime by dragging mouse.


Processing: image_003.jpg

✅ Selected area: 180,186 to 228,227
Save crop as:
  'b' -> backgrounds
  '1' -> 1stdoor
  '2' -> 2sdoor
  's' -> stove
  't' -> toilet
  'k' -> sink
  'c' -> closet
Other:
  SPACE -> next image
  q     -> quit
Crop again anytime by dragging mouse.

💾 Saved 1STDOOR: 1stdoor_01.png

✅ Selected area: 229,154 to 263,195
Save crop as:
  'b' -> backgrounds
  '1' -> 1stdoor
  '2' -> 2sdoor
  's' -> stove
  't' -> toilet
  'k' -> sink
  'c' -> closet
Other:
  SPACE -> next image
  q     -> quit
Crop again anytime by dragging mouse.

💾 Saved 1STDOOR: 1stdoor_02.png

✅ Selected area: 275,154 to 308,192
Save crop as:
  'b' -> backgrounds
  '1' -> 1stdoor
  '2' -> 2sdoor
  's' -> stove
  't' -> to